In [16]:
# ! pip install python-dotenv
# ! pip install langchain --upgrade --no-cache-dir
! pip install wandb --upgrade --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 368.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=c6c220c89ae33bb1d25761d926c629c5cf2ec102fc84caf1ae9dcf411d406bdc
  Stored in directory: /tmp/pip-ephem-wheel-cache-w2bmt6xz/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [67]:
%load_ext autoreload
! ls -alh /var/genie-in-the-box/src/lib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
total 32K
drwxr-xr-x  3 1000 1000 4.0K Sep  7 19:16 .
drwxr-xr-x 12 1000 1000 4.0K Sep  7 02:42 ..
-rw-r--r--  1 1000 1000    0 Jun 13 16:35 __init__.py
drwxr-xr-x  2 1000 1000 4.0K Sep  7 19:16 __pycache__
-rw-rw-r--  1 1000 1000  11K Sep  6 00:35 util.py
-rw-rw-r--  1 1000 1000 2.3K Sep  7 19:16 util_langchain.py
-rw-rw-r--  1 1000 1000 3.1K Sep  6 00:35 util_stopwatch.py


In [109]:
%autoreload

import sys
import os
import json

os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
# wandb documentation to configure wandb using env variables
# https://docs.wandb.ai/guides/track/advanced/environment-variables
# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "langchain-dataframe-agent"

path = "/var/genie-in-the-box/src/lib"
if path not in sys.path:
    sys.path.append( path )
else:
    print( f"[{path}] already in sys.path" ) 
    
print( sys.path )

import util as du
path = "/var/genie-in-the-box/src"
du.add_to_path( path )

import util_stopwatch as sw
import util_langchain as ulc
import genie_client as gc

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent

import pandas as pd


from IPython.display import display, Markdown


[/var/genie-in-the-box/src/lib] already in sys.path
['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/var', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/var/genie-in-the-box/src/lib', '/var/genie-in-the-box/src']
Path [/var/genie-in-the-box/src] already in sys.path


In [13]:
# llm = ChatOpenAI( model_name="gpt-4", temperature=0.0 )
langchain.debug = True
langchain.verbose = True

llm    = ChatOpenAI( model_name="gpt-4-0613", temperature=0.0 )
memory = ConversationBufferMemory( memory_key="chat_history", return_messages=True, verbose=True )
raw_df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )

df_agent        = create_pandas_dataframe_agent( llm, raw_df, verbose=True )
df_agent.memory = memory

In [9]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history')

In [18]:
df_agent.run( "What day is today?" )

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Streaming LangChain activity to W&B at https://wandb.ai/ricardo-felipe-ruiz/langchain-dataframe-agent/runs/lz5qpkzj
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "What day is today?",
  "chat_history": []
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "What day is today?",
  "chat_history": [],
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think

'Today is 2023-09-07.'

In [53]:
pandas_prompt_template = '''
You are working with a pandas dataframe in Python. The name of the dataframe is `df`.

This is the result of `print(df.head())`:

"start_date","end_date","start_time","end_time","event_type","recurrent","recurrence_interval","priority_level","name","relationship","description_who_what_where"
"2023-07-01","2023-07-04","00:00","23:59","Concert",False,"","none","Jenny","coworker","Concert of Jenny at the city center"
"2023-07-01","2023-07-01","05:25","17:22","TODO",False,"","highest","Gregorio","friend","Send out invitations for the party for Gregorio"
"2023-07-01","2023-07-01","13:27","01:59","Appointment",False,"","high","Leroy Ruiz","father","Appointment with Leroy Ruiz at the clinic"
"2023-07-03","2023-07-04","00:00","23:59","Subscription",True,"1 year","highest","Leroy Ruiz","father","Renewal of Leroy Ruiz's subscription"
"2023-07-03","2023-07-04","00:00","23:59","Anniversary",True,"4 week","none","Juan","neighbor","Juan's anniversary celebration at the park"
"2023-07-04","2023-07-04","00:00","23:59","Anniversary",True,"4 week","medium","Leroy Ruiz","father","Leroy Ruiz's anniversary celebration at the park"

I want you to do three things for me:
1) Thoughts: Think out loud about what I'm asking you and how you're going to translate the question below into working python code that can be used to query `df` for the answer.
2) Code: A verbatim list of code, one line of code per item on the list
3) Explanation: Briefly and succinctly explain your code in plain English.

Format: return your response as a JSON object in the following fields:
{{
    "thoughts": "your thoughts",
    "code": [
        "count = df.shape[ 0 ]",
        "print( count )"
    ],
    "explanation": "your explanation"
}}

Begin!

Question: {question}
'''

# question = "How many records are in this dataset?"
question = "Do I have any events today?"
# pandas_prompt = pandas_prompt_template.format( question=question )
# pandas_prompt

In [54]:
from langchain import PromptTemplate
from langchain import LLMChain

prompt = PromptTemplate(
    template=pandas_prompt_template,
    input_variables=["question"]
)
llm_chain = LLMChain(
   prompt=prompt,
   llm=llm
)

timer = sw.Stopwatch( "Running pandas prompt..." )
my_response = llm_chain.run( question=question )
timer.print( "Done!" )

Running pandas prompt...
[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "question": "Do I have any events today?"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\n\nThis is the result of `print(df.head())`:\n\n\"start_date\",\"end_date\",\"start_time\",\"end_time\",\"event_type\",\"recurrent\",\"recurrence_interval\",\"priority_level\",\"name\",\"relationship\",\"description_who_what_where\"\n\"2023-07-01\",\"2023-07-04\",\"00:00\",\"23:59\",\"Concert\",False,\"\",\"none\",\"Jenny\",\"coworker\",\"Concert of Jenny at the city center\"\n\"2023-07-01\",\"2023-07-01\",\"05:25\",\"17:22\",\"TODO\",False,\"\",\"highest\",\"Gregorio\",\"friend\",\"Send out invitations for the party for Gregorio\"\n\"2023-07-01\",\"2023-07-01\",\"13:27\",\"01:59\",\"Appointment\",False,\"\",\"high\",\"Leroy Ruiz\",\"father\",\"Appointment wi

In [55]:
response_dict = json.loads( my_response )
response_dict

{'thoughts': "To answer this question, I need to check if today's date falls between the 'start_date' and 'end_date' for any events in the dataframe. I will use the pandas function 'between' to check if today's date is within the range of 'start_date' and 'end_date'. I will then count the number of rows that meet this condition, which will give me the number of events today. If the count is greater than zero, then there are events today.",
 'code': ['import pandas as pd',
  "today = pd.to_datetime('today').date()",
  "events_today = df[(df['start_date'] <= today) & (df['end_date'] >= today)]",
  'count = events_today.shape[0]',
  'print(count > 0)'],
 'explanation': "First, I import the pandas library. Then, I get today's date using the 'to_datetime' function from pandas and convert it to just the date with no time. Next, I create a new dataframe 'events_today' that only includes rows from 'df' where today's date is between the 'start_date' and 'end_date'. I then get the number of rows

In [56]:
response_dict[ "code" ]

['import pandas as pd',
 "today = pd.to_datetime('today').date()",
 "events_today = df[(df['start_date'] <= today) & (df['end_date'] >= today)]",
 'count = events_today.shape[0]',
 'print(count > 0)']

In [71]:
ulc.assemble_and_run_solution( response_dict[ "code" ], path=du.get_project_root() + "/src/conf/long-term-memory/events.csv" )


{'return_code': 1,
 'response': 'ERROR:\nTraceback (most recent call last):\n  File "/var/genie-in-the-box/io/code.py", line 5, in <module>\n    events_today = df[(df[\'start_date\'] <= today) & (df[\'end_date\'] >= today)]\n  File "/usr/local/lib/python3.10/dist-packages/pandas/core/ops/common.py", line 72, in new_method\n    return method(self, other)\n  File "/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py", line 54, in __le__\n    return self._cmp_method(other, operator.le)\n  File "/usr/local/lib/python3.10/dist-packages/pandas/core/series.py", line 6243, in _cmp_method\n    res_values = ops.comparison_op(lvalues, rvalues, op)\n  File "/usr/local/lib/python3.10/dist-packages/pandas/core/ops/array_ops.py", line 287, in comparison_op\n    res_values = comp_method_OBJECT_ARRAY(op, lvalues, rvalues)\n  File "/usr/local/lib/python3.10/dist-packages/pandas/core/ops/array_ops.py", line 75, in comp_method_OBJECT_ARRAY\n    result = libops.scalar_compare(x.ravel(), y, op)\

In [47]:
timer = sw.Stopwatch( "Running pandas prompt..." )
question = "Do I have any events today?"
response = llm_chain.run( question=question )
timer.print( "Done!" )

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


Running pandas prompt...
[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "question": "Do I have any events today?"
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\n\nThis is the result of `print(df.head())`:\n\n\"start_date\",\"end_date\",\"start_time\",\"end_time\",\"event_type\",\"recurrent\",\"recurrence_interval\",\"priority_level\",\"name\",\"relationship\",\"description_who_what_where\"\n\"2023-07-01\",\"2023-07-04\",\"00:00\",\"23:59\",\"Concert\",False,\"\",\"none\",\"Jenny\",\"coworker\",\"Concert of Jenny at the city center\"\n\"2023-07-01\",\"2023-07-01\",\"05:25\",\"17:22\",\"TODO\",False,\"\",\"highest\",\"Gregorio\",\"friend\",\"Send out invitations for the party for Gregorio\"\n\"2023-07-01\",\"2023-07-01\",\"13:27\",\"01:59\",\"Appointment\",False,\"\",\"high\",\"Leroy Ruiz\",\"father\",\"Appointment wi

In [48]:
response_dict = json.loads( response )
response_dict[ "code" ]

['import pandas as pd',
 'from datetime import date',
 "df['start_date'] = pd.to_datetime(df['start_date'])",
 "df['end_date'] = pd.to_datetime(df['end_date'])",
 'today = pd.to_datetime(date.today())',
 "events_today = df[(df['start_date'] <= today) & (df['end_date'] >= today)]",
 'if not events_today.empty:',
 "    print('Yes, you have events today.')",
 'else:',
 "    print('No, you do not have any events today.')"]

In [77]:
agent_response = df_agent.run( "Do I have any events today?  You must verify that your code runs to completion, once it does, return your answer as a JSON string containg three fields: 1) answer: Your answer to the question in informal, conversational English, 2) code: The Python code you generated in python list form, one line of code per item in the list, and 3) explanation: A brief explanation as to how the code works" )

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/

In [78]:
agent_response_dict = json.loads( agent_response )
agent_response_dict

{'answer': 'Yes, there are events today.',
 'code': ['import datetime',
  'today = datetime.date.today()',
  "events_today = df.loc[(df['start_date'] <= str(today)) & (df['end_date'] >= str(today))]",
  'events_today.empty'],
 'explanation': "The code first imports the datetime library to get today's date. It then checks if there are any events in the dataframe that have a start_date or end_date that is equal to today's date. If the dataframe of events today is not empty, then there are events today."}

In [80]:
ulc.assemble_and_run_solution( agent_response_dict[ "code" ], path=du.get_project_root() + "/src/conf/long-term-memory/events.csv" )

{'return_code': 0, 'response': 'False'}

In [88]:
question_plus_coda_template = """
{event_query}  

Hint: An event that I have today may have started before today and may end tomorrow or next week, so be careful how you filter on dates.

You must verify that your code runs to completion, once it does, return your answer as a JSON string containing three fields: 
1) answer: Your answer to the question in informal, conversational English, 
2) code: The Python code you generated in python list form, one line of code per item in the list, and 
3) explanation: A brief explanation as to how the code works"""

question = question_plus_coda_template.format( event_query="What events do I have today?" )
question


'\nWhat events do I have today?  \n\nHint: An event that I have today may have started before today and may end tomorrow or next week, so be careful how you filter on dates.\n\nYou must verify that your code runs to completion, once it does, return your answer as a JSON string containing three fields: \n1) answer: Your answer to the question in informal, conversational English, \n2) code: The Python code you generated in python list form, one line of code per item in the list, and \n3) explanation: A brief explanation as to how the code works'

In [89]:
timer = sw.Stopwatch( "Running pandas prompt..." )
agent_response = df_agent.run( question )
timer.print( "Done!" )

agent_response_dict = json.loads( agent_response )
agent_response_dict

Running pandas prompt...
[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [98]:
import re

# Slicing gets us a copy of the string
malformed_string = agent_response[:]
print(malformed_string)

# Extract the JSON data using regular expression
match = re.search(r"\{.*\}", malformed_string)
if match:
    json_data = match.group()
    print(json_data)
else:
    print("No JSON data found in the string.")

```json
{
  "answer": "You have one event today. It is a workshop with John on AGI.",
  "code": [
    "from datetime import datetime",
    "today = datetime.now().date()",
    "today = pd.to_datetime(today)",
    "df['start_date'] = pd.to_datetime(df['start_date'])",
    "df['end_date'] = pd.to_datetime(df['end_date'])",
    "events_today = df[(df['start_date'] <= today) & (df['end_date'] >= today)]",
    "events_today"
  ],
  "explanation": "The code first gets today's date and converts it to a pandas Timestamp object. It then converts the 'start_date' and 'end_date' columns in the dataframe to datetime. Finally, it filters the dataframe to show only the rows where the start date is less than or equal to today's date and the end date is greater than or equal to today's date. The result is a dataframe that contains only the events that are happening today."
}
```
No JSON data found in the string.


In [126]:
# Kludgy way to strip off anything before and after the opening and closing curly brackets
stripped_response = "{" + agent_response.split( "{" )[ 1 ].split( "}" )[ 0 ] + "}"

json.loads( stripped_response )

IndexError: list index out of range

In [127]:
stripped_response = agent_response.strip( "```" ).strip( "json" ).strip( "\n" )
agent_response_dict = json.loads( stripped_response )
agent_response_dict

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [101]:
agent_response_dict[ "code" ]

['from datetime import datetime',
 'today = datetime.now().date()',
 'today = pd.to_datetime(today)',
 "df['start_date'] = pd.to_datetime(df['start_date'])",
 "df['end_date'] = pd.to_datetime(df['end_date'])",
 "events_today = df[(df['start_date'] <= today) & (df['end_date'] >= today)]",
 'events_today']

In [106]:
response = ulc.assemble_and_run_solution( agent_response_dict[ "code" ], path=du.get_project_root() + "/src/conf/long-term-memory/events.csv" )

for line in response[ "response" ].split( "\n" ):
    print( line )

start_date   end_date  ... relationship          description_who_what_where
195 2023-08-31 2023-09-07  ...     coworker  Exciting workshop with John on AGI

[1 rows x 11 columns]


In [115]:
genie_client = gc.GenieClient()
genie_client.ask_chat_gpt_text( response[ "response" ], preamble="This is output from a query on a pandas dataframe. Rephrase this output in conversational English in such a way that answers the question: what events do I have today?" )

         self.project_root [/var/genie-in-the-box]
translation-dictionary.map [/var/genie-in-the-box/src/conf/translation-dictionary.map]
Asking ChatGPT [gpt-4-0613]...Done! in 3,909 ms


'You have an exciting workshop with your coworker John on AGI today. The event starts on August 31, 2023, and ends on September 7, 2023.'

In [124]:
agent_response = df_agent.run( "What is tomorrow's date?" )

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/

In [125]:
df_agent.run( "What day was yesterday?" )

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/

'Yesterday was 2023-09-06.'

In [180]:
df_agent.run( "Today is 2023-07-27. What events do i have today? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events that have the same start_date as today.

Action: python_repl_ast
Action Input: df[df['start_date'] == '2023-07-27']
Observation:     start_date    end_date  duration start_time end_time    event_type  \
71  2023-07-27  2023-07-27         0      00:00    23:59   Anniversary   
72  2023-07-27  2023-07-30         3      00:00    23:59  Subscription   
73  2023-07-27  2023-07-28         1      05:03    06:50          TODO   
74  2023-07-27  2023-07-27         0      22:07    03:01          TODO   

    recurrent recurrence_interval                      description  \
71       True              2 week  Gregorio (friend)'s anniversary   
72       True               3 day          This is a subscription.   
73      False                 NaN                  This is a todo.   
74      False                 NaN                  This is a todo.   

   priority_level  
71           high  
72    

'The events I have today are:\n- Anniversary (recurrent event)\n- Subscription (recurrent event)\n- TODO (non-recurrent event)\n- TODO (non-recurrent event)'

In [189]:
df_agent.run( "Today is 2023-07-27. What events do i have today? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events that have the same start_date as today.

Action: python_repl_ast
Action Input: df[df['start_date'] == '2023-07-27']
Observation:     start_date    end_date  duration start_time end_time    event_type  \
71  2023-07-27  2023-07-27         0      00:00    23:59   Anniversary   
72  2023-07-27  2023-07-30         3      00:00    23:59  Subscription   
73  2023-07-27  2023-07-28         1      05:03    06:50          TODO   
74  2023-07-27  2023-07-27         0      22:07    03:01          TODO   

    recurrent recurrence_interval                      description  \
71       True              2 week  Gregorio (friend)'s anniversary   
72       True               3 day          This is a subscription.   
73      False                 NaN                  This is a todo.   
74      False                 NaN                  This is a todo.   

   priority_level  
71           high  
72    

'The events I have today are:\n- Anniversary (recurrent event)\n- Subscription (recurrent event)\n- TODO (non-recurrent event)\n- TODO (non-recurrent event)'

In [179]:
raw_df[ raw_df[ 'start_date' ] == '2023-07-27' ][ [ 'event_type', 'start_time' ] ]

,event_type,start_time
73,TODO,05:03
74,TODO,22:07
71,Anniversary,00:00
72,Subscription,00:00


In [181]:
response = df_agent.run( "What events do i have today, when do they start? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events that have the start_date equal to today's date. Then I can select the event_type and start_time columns to see what events I have today and when they start.

Action: python_repl_ast
Action Input: df[df['start_date'] == pd.to_datetime('today').date()][['event_type', 'start_time']]
Observation: Empty DataFrame
Columns: [event_type, start_time]
Index: []
Thought:

OutputParserException: Could not parse LLM output: `There are no events today.`

In [182]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='What day is today? Use tool python_repl_ast to respond.', additional_kwargs={}, example=False), AIMessage(content='2023-07-27', additional_kwargs={}, example=False), HumanMessage(content='How many records are in this dataset?', additional_kwargs={}, example=False), AIMessage(content='200', additional_kwargs={}, example=False), HumanMessage(content='What day is tomorrow? Use tool python_repl_ast', additional_kwargs={}, example=False), AIMessage(content="Tomorrow's date is stored in the variable `tomorrow`.", additional_kwargs={}, example=False), HumanMessage(content='What date is tomorrow? Use tool python_repl_ast', additional_kwargs={}, example=False), AIMessage(content="Tomorrow's date is 2023-07-03.", additional_kwargs={}, example=False), HumanMessage(content="Today is 2023-07-27. What is tomorrow's date? Use tool python_repl_ast", additional_kwargs={}, example=False), AIMessage(content="Tomorrow'

In [183]:
df_agent.run( "What have we discussed thus far? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `Thought: We have discussed the structure and contents of the dataframe `df`.

Action: None
`

In [184]:
df_agent.run( "Today is July 27th, 2023. Do I have any events today? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: To check if there are any events today, I need to compare the current date with the start_date and end_date columns in the dataframe.

Action: Use python_repl_ast
Action Input: `df[(df['start_date'] <= '2023-07-27') & (df['end_date'] >= '2023-07-27')]`
Observation: Use python_repl_ast is not a valid tool, try another one.
Thought:I can use the `query` method in pandas to filter the dataframe based on the conditions.

Action: Use python_repl_ast
Action Input: `df.query("'2023-07-27' >= start_date and '2023-07-27' <= end_date")`
Observation: Use python_repl_ast is not a valid tool, try another one.
Thought:I can use the `loc` method in pandas to filter the dataframe based on the conditions.

Action: Use python_repl_ast
Action Input: `df.loc[(df['start_date'] <= '2023-07-27') & (df['end_date'] >= '2023-07-27')]`
Observation: Use python_repl_ast is not a valid tool, try another one.
Thought:I can use the `loc` method in pandas to filter the 

KeyboardInterrupt: 

In [185]:
df_agent.run( "Today is July 27th, 2023. What kind of event do I have today? Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: To find out what kind of event I have today, I need to compare the current date with the start_date and end_date columns in the dataframe.

Action: Use the pandas `datetime` module to get the current date and compare it with the start_date and end_date columns.

Action Input: 
```python_repl_ast
import pandas as pd
from datetime import datetime

current_date = datetime(2023, 7, 27).date()
df['start_date'] = pd.to_datetime(df['start_date']).dt.date
df['end_date'] = pd.to_datetime(df['end_date']).dt.date
event_today = df[(df['start_date'] <= current_date) & (df['end_date'] >= current_date)]
event_today['event_type']
```

Observation: Use the pandas `datetime` module to get the current date and compare it with the start_date and end_date columns. is not a valid tool, try another one.
Thought:

OutputParserException: Could not parse LLM output: `I need to use the `python_repl_ast` tool to execute the code and get the result.`

In [186]:
response = df_agent.run( "What kind of event do I have for July 27th, 2023? Tell me their description and what time they start. Use markdown to format the output. Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: To find the event for July 27th, 2023, I need to filter the dataframe based on the start_date column.

Action: Use the pandas loc function to filter the dataframe.

Action Input: `df.loc[df['start_date'] == '2023-07-27']`

Observation: Use the pandas loc function to filter the dataframe. is not a valid tool, try another one.
Thought:The pandas loc function is not a valid tool in this case. I should try using the pandas query function instead.

Action: Use the pandas query function to filter the dataframe.

Action Input: `df.query("start_date == '2023-07-27'")`

Observation: Use the pandas query function to filter the dataframe. is not a valid tool, try another one.
Thought:The pandas query function is also not a valid tool in this case. I should try using boolean indexing instead.

Action: Use boolean indexing to filter the dataframe.

Action Input: `df[df['start_date'] == '2023-07-27']`

Observation: Use boolean indexing to filter the d

KeyboardInterrupt: 

In [91]:
Markdown( response )

|    | event_type   | description             | start_time   |
|---:|:-------------|:------------------------|:-------------|
|  0 | Performance  | This is a performance.  | 00:00        |
|  1 | Interview    | This is a interview.    | 15:31        |

In [84]:
response = df_agent.run( "What kind of event do I have for July 27th, 2023? Tell me their description and what time they start. Use tool python_repl_ast" )



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events on July 27th, 2023 and then select the columns for event type, description, and start time.

Action: Use the `loc` function to filter the dataframe and select the desired columns.

Action Input: `df.loc[df['date'] == '2023-07-27', ['event_type', 'description', 'start_time']]`

Observation: Use the `loc` function to filter the dataframe and select the desired columns. is not a valid tool, try another one.
Thought:I need to use the `python_repl_ast` tool to execute the command.

Action: Use the `python_repl_ast` tool.

Action Input: `python_repl_ast`

Observation: Use the `python_repl_ast` tool. is not a valid tool, try another one.
Thought:I need to use the `python_repl_ast` tool to execute the command.

Action: Use the `python_repl_ast` tool.

Action Input: `python_repl_ast`

Observation: Use the `python_repl_ast` tool. is not a valid tool, try another one.
Thought:I need to use the `

In [187]:
response = df_agent.run( "Show me the events I have on July 28th, 2023. Use tool python_repl_ast" )
response



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events on July 28th, 2023.

Action: Use the `loc` function to filter the dataframe.

Action Input: `df.loc[df['start_date'] == '2023-07-28']`

Observation: Use the `loc` function to filter the dataframe. is not a valid tool, try another one.
Thought:

OutputParserException: Could not parse LLM output: `I need to use the `python_repl_ast` tool to execute the command.`

In [188]:
response = df_agent.run( "What events do I have for July 28th, 2023. Use tool python_repl_ast" )
response



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include events that occur on July 28th, 2023.

Action: Use the `loc` function to filter the dataframe based on the `start_date` column.

Action Input: `df.loc[df['start_date'] == '2023-07-28']`

Observation: Use the `loc` function to filter the dataframe based on the `start_date` column. is not a valid tool, try another one.
Thought:I need to use the `query` function instead of the `loc` function to filter the dataframe based on the `start_date` column.

Action: Use the `query` function to filter the dataframe based on the `start_date` column.

Action Input: `df.query("start_date == '2023-07-28'")`

Observation: Use the `query` function to filter the dataframe based on the `start_date` column. is not a valid tool, try another one.
Thought:I can use the `loc` function to filter the dataframe based on the `start_date` column.

Action: Use the `loc` function to filter the dataframe based on the `start_

'Agent stopped due to iteration limit or time limit.'

In [44]:
df_agent.agent.llm_chain.prompt.to_json()[ "kwargs" ][ "template" ]

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df_head}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'

In [36]:
dir(df_agent.agent.llm_chain.prompt.partial_variables.items)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__text_signature__']

In [11]:
agent_json = df_agent.to_json()
agent_json.keys()

dict_keys(['lc', 'type', 'id', 'repr'])

In [28]:
agent_json[ 'repr' ]

"AgentExecutor(memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history'), callbacks=None, callback_manager=None, verbose=True, tags=None, metadata=None, agent=ZeroShotAgent(llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=PromptTemplate(input_variables=['input', 'agent_scratchpad'], output_parser=None, partial_variables={'df_head': '|    | start_date   | end_date   | start_time   | end_time   | event_type   | recurrent   |   recurrence_interval | priority_level   | name       | relationship   | description_who_what_where                      |\\n|---:|:-------------|:-----------|:-------------|:-----------|:-------------|:------------|----------------------:|:-----------------|:-----------|:---------------|:------------------------------------------------|\\n|  0 | 2023-07-01   | 